In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install hopsworks

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import xgboost as xgb


In [4]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split

In [5]:
%cd /content/drive/MyDrive/jim

/content/drive/MyDrive/jim


In [6]:
from functions import *

In [15]:
def data_encoder(X):
    X.drop(columns=['date'], inplace=True)
    category_cols = ['conditions','aqi']
    col_names=[]
    col_names=X.columns.values
    for col_name in col_names:
      if col_name not in category_cols:
        X[col_name]=StandardScaler().fit_transform(X[[col_name]])
    X['conditions'] = OrdinalEncoder().fit_transform(X[['conditions']])
    return X

In [24]:
data=pd.read_csv("/content/drive/MyDrive/jim/training.csv")
print(len(data))

3293


In [25]:
data=data[data.aqi<200]
print(len(data))

3025


In [26]:
data_encoder(data)

#data.show(5)

,aqi,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,...,snow,snowdepth,windspeed,winddir,cloudcover,visibility,solarradiation,solarenergy,uvindex,conditions
0,136,-0.733652,-0.771028,-0.671537,-0.634675,-0.673159,-0.682797,-1.381769,-1.747479,-0.199685,...,-0.037823,-0.094163,1.163434,0.978166,-1.329522,1.159628,-0.976255,-0.970700,-0.703621,0.0
2,127,-0.943986,-0.857860,-0.965169,-0.899350,-0.929923,-0.940973,-1.133212,-1.060726,-0.199685,...,-0.037823,-0.094163,-0.475516,-1.584121,-1.255464,2.177810,-0.962905,-0.970700,-0.703621,0.0
4,168,-1.206904,-1.292022,-1.303291,-1.234087,-1.458555,-1.363444,-0.969875,-0.073836,-0.199685,...,-0.037823,-0.094163,-0.284472,-1.507922,-1.274953,-0.435524,-0.955624,-0.928573,-0.703621,0.0
6,178,-1.294543,-1.118357,-1.223209,-1.257440,-1.201791,-1.253915,-1.047993,-0.323102,-0.199685,...,-0.037823,-0.094163,-0.284472,-1.536382,0.572582,-0.435524,-1.311195,-1.307716,-1.167539,2.0
7,65,-1.645100,-1.378854,-1.516841,-1.880206,-1.458555,-1.660738,-1.822070,-1.630477,-0.199685,...,-0.037823,-0.094163,1.163434,1.455554,-1.029395,6.589931,-0.859753,-0.858361,-0.703621,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3288,38,-1.469821,-1.465686,-1.481250,-1.288579,-1.458555,-1.519915,-1.388871,-0.974246,-0.199685,...,-0.037823,-0.094163,-0.284472,-1.429888,-0.191378,0.854173,-1.029651,-1.040912,-0.703621,2.0
3289,62,-1.513641,-1.552519,-1.677004,-1.366424,-1.654904,-1.574679,-1.360464,-0.521497,-0.199685,...,-0.037823,-0.094163,-0.646449,1.498702,-0.475914,0.616597,-1.134016,-1.139208,-1.167539,2.0
3290,111,-1.294543,-1.639351,-1.623617,-1.132887,-1.549178,-1.433856,-1.147416,-0.022965,-0.199685,...,-0.037823,-0.094163,-1.008426,0.642158,-1.228180,0.107507,-1.035719,-1.026869,-0.703621,0.0
3291,170,-1.294543,-1.726183,-1.668106,-1.132887,-1.730423,-1.480797,-0.976977,0.623092,-0.199685,...,-0.037823,-0.094163,-1.008426,-1.639204,-0.772143,-0.401584,-1.019942,-1.012827,-0.703621,0.0


In [27]:
train_data, test= train_test_split(data, test_size=0.2, random_state=42)

In [28]:
X = data.fillna(0)
y = X.pop("aqi")

In [29]:
X_train = train_data.fillna(0)
y_train = X_train.pop("aqi")

In [30]:
X_test = test.fillna(0)
y_test = X_test.pop("aqi")

In [31]:
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
import xgboost as xgb

# Create the parameter grid for the base models
param_grid_svr = {
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.1, 0.2, 0.3, 0.4],
    'gamma': [0.1, 0.2, 0.3, 0.4,0.5]
}
param_grid_xgb = {
    'max_depth': [3, 4, 5, 6,7,8],
    'learning_rate': [0.1, 0.2, 0.3, 0.4],
    'n_estimators': [100,150, 200, 250,300,350, 400]
}

# Create the parameter grid for the ensemble model
param_grid_ensemble = {
    'cv': [3, 5, 7]
}

# Create the base models
svr = SVR(kernel='rbf')
xgb_model = xgb.XGBRegressor()

# Create the stacking regressor
ensemble_model = StackingRegressor(estimators=[('svr', svr), ('xgb', xgb_model)])

# Create the grid search models
grid_search_svr = GridSearchCV(svr, param_grid_svr, cv=5, verbose=2, n_jobs=-1)
grid_search_xgb = GridSearchCV(xgb_model, param_grid_xgb, cv=5, verbose=2, n_jobs=-1)
grid_search_ensemble = GridSearchCV(ensemble_model, param_grid_ensemble, cv=5, verbose=2, n_jobs=-1)

# Fit the grid search models to the data
grid_search_svr.fit(X_train, y_train)
grid_search_xgb.fit(X_train, y_train)
grid_search_ensemble.fit(X_train, y_train)

# Print the best parameters and best scores for the base models
print(grid_search_svr.best_params_)
print(grid_search_svr.best_score_)
print(grid_search_xgb.best_params_)
print(grid_search_xgb.best_score_)

# Print the best parameters and best score for the ensemble model
print(grid_search_ensemble.best_params_)
print(grid_search_ensemble.best_score_)


Fitting 5 folds for each of 80 candidates, totalling 400 fits
Fitting 5 folds for each of 168 candidates, totalling 840 fits
[19:39:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 3 candidates, totalling 15 fits
[19:39:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:39:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:39:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:39:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:39:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:39:51] WARNING: /workspace/src/objective/regressi

In [32]:
best_model=grid_search_ensemble

In [33]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
y_pred= best_model.predict(X_test)
print(max(y_pred-y_test), min(y_pred-y_test))
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(rmse)

69.96892928557564 -74.41246280644782
19.146468585862046


In [34]:
y_pred= best_model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
print(rmse)

17.98748822119084


In [35]:
y_pred= best_model.predict(X)
mae = mean_absolute_error(y, y_pred)
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)
print(rmse)

18.225181422083676


In [36]:
import hopsworks

project = hopsworks.login() 

fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5333


Connected. Call `.close()` to terminate connection gracefully.


In [37]:
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [41]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X)
output_schema = Schema(y)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

{'input_schema': {'columnar_schema': [{'name': 'tempmax', 'type': 'float64'},
   {'name': 'tempmin', 'type': 'float64'},
   {'name': 'temp', 'type': 'float64'},
   {'name': 'feelslikemax', 'type': 'float64'},
   {'name': 'feelslikemin', 'type': 'float64'},
   {'name': 'feelslike', 'type': 'float64'},
   {'name': 'dew', 'type': 'float64'},
   {'name': 'humidity', 'type': 'float64'},
   {'name': 'precip', 'type': 'float64'},
   {'name': 'precipprob', 'type': 'float64'},
   {'name': 'precipcover', 'type': 'float64'},
   {'name': 'snow', 'type': 'float64'},
   {'name': 'snowdepth', 'type': 'float64'},
   {'name': 'windspeed', 'type': 'float64'},
   {'name': 'winddir', 'type': 'float64'},
   {'name': 'cloudcover', 'type': 'float64'},
   {'name': 'visibility', 'type': 'float64'},
   {'name': 'solarradiation', 'type': 'float64'},
   {'name': 'solarenergy', 'type': 'float64'},
   {'name': 'uvindex', 'type': 'float64'},
   {'name': 'conditions', 'type': 'float64'}]},
 'output_schema': {'columna

In [42]:
import joblib

joblib.dump(best_model, 'model.pkl')

['model.pkl']

In [43]:
model = mr.sklearn.create_model(
    name="aqi_ensemble",
    metrics={"rmse": "19.15"},
    description="ensembled aqi prediction model.",
    input_example=X.sample().to_numpy(),
    model_schema=model_schema
)

model.save('model.pkl')

  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://c.app.hopsworks.ai:443/p/5333/models/aqi_ensemble/3


Model(name: 'aqi_ensemble', version: 3)